In [1]:
import csv
import pandas as pd 
from sklearn.feature_selection import mutual_info_regression 
import seaborn as sns 
import matplotlib.pyplot as plt 
import random 
from mpl_toolkits.mplot3d import Axes3D 
import pylab
import numpy as np 
from sklearn.ensemble import RandomForestRegressor 
from xgboost import XGBRegressor
import xgboost as xgb 
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning) 
pd.set_option('display.max_rows', 1000) 
pd.set_option('display.max_columns', 1000) 
from sklearn.cluster import KMeans 
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import *
from sklearn import ensemble
from sklearn.linear_model import *
from sklearn.neighbors import KNeighborsRegressor
from mpl_toolkits.mplot3d import Axes3D 
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVR
from math import radians, cos, sin, asin, sqrt
import mpu
from sklearn.metrics import r2_score
import gc

from sklearn.metrics import make_scorer

def mdape(y_true,y_pred, **kwargs):
    mdape = abs((y_pred-y_true)/y_true)
    return mdape
my_scorer = make_scorer(mdape, greater_is_better=False)

In [2]:
df = pd.read_csv(r'rent_new.csv', sep=';')


In [3]:

def basic_details(df):
    """returns basic statistics for dataset - % of zero, nan and unique values"""
    b = pd.DataFrame()
    b['Missing value, %'] = round(df.isnull().sum()/df.shape[0]*100,4)
    b['Zero value, %'] = round(df.isin([0]).sum() /df.shape[0]*100,4)
    b['N unique value'] = df.nunique()
    b['dtype'] = df.dtypes
    return b

def plotting(x):

    '''boxplotting & distplotting'''
    f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, 
                                        gridspec_kw={"height_ratios": (.15, .85)})

    sns.boxplot(x, y=None,  ax=ax_box)
    sns.distplot(x, ax=ax_hist)

    ax_box.set(yticks=[])
    sns.despine(ax=ax_hist)
    sns.despine(ax=ax_box, left=True)
    return 

def make_mi_scores(X, y):
    """mutual info-scores calculator"""
    mi_scores = mutual_info_regression(X, y)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

def reduce_memory_usage(df):
    """ The function will reduce memory of dataframe
    Note: Apply this function after removing missing value"""
    intial_memory = df.memory_usage().sum()/1024**2
    print('Intial memory usage:',intial_memory,'MB')
    for col in df.columns:
        mn = df[col].min()
        mx = df[col].max()
        if df[col].dtype != object:            
            if df[col].dtype == int:
                if mn >=0:
                    if mx < np.iinfo(np.uint8).max:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < np.iinfo(np.uint16).max:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < np.iinfo(np.uint32).max:
                        df[col] = df[col].astype(np.uint32)
                    elif mx < np.iinfo(np.uint64).max:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
            if df[col].dtype == float:
                df[col] =df[col].astype(np.float32)
    
    red_memory = df.memory_usage().sum()/1024**2
    print('Memory usage after complition: ',red_memory,'MB')

def find_longest_word(word_list):  
    longest_word =  max(word_list, key=len)
    return longest_word


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563684 entries, 0 to 563683
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   RealtorListingId   563684 non-null  int64  
 1   RealtorPropertyId  563684 non-null  object 
 2   DateUpdateListing  563684 non-null  object 
 3   Price              563684 non-null  int64  
 4   Latitude           563684 non-null  float64
 5   Longitude          563684 non-null  float64
 6   PostalCode         563684 non-null  int64  
 7   StateCode          563684 non-null  object 
 8   City               563684 non-null  object 
 9   Address            561177 non-null  object 
 10  Neighborhood       323034 non-null  object 
 11  PropertyType       563684 non-null  object 
 12  YearBuilt          277271 non-null  float64
 13  Beds               563684 non-null  int64  
 14  Baths              563684 non-null  int64  
 15  BuildingSize       139836 non-null  float64
 16  lo

In [ ]:
basic_details(df)

,"Missing value, %","Zero value, %",N unique value,dtype
RealtorListingId,0.0000,0.0000,449634,int64
RealtorPropertyId,0.0000,0.0000,431894,object
DateUpdateListing,0.0000,0.0000,206404,object
Price,0.0000,0.0000,7504,int64
Latitude,0.0000,0.0000,306642,float64
Longitude,0.0000,0.0000,308454,float64
PostalCode,0.0000,0.0000,14199,int64
StateCode,0.0000,0.0000,52,object
City,0.0000,0.0000,8262,object
Address,0.4448,0.0000,422863,object


In [ ]:
df = df[(df.Price<df.Price.quantile(0.98)) & (df.Price>df.Price.quantile(0.02))]

df = df[((df.Longitude<-50)&(df.Longitude>-140))]
df.Longitude = abs(df.Longitude)
df = df[(df.Latitude<50)]

df.Neighborhood = df.Neighborhood.fillna(df.City)

df.City = df.City+'_'+df.StateCode

df.DateUpdateListing = pd.to_datetime(df.DateUpdateListing)
df['YearRent'] = df['DateUpdateListing'].dt.year
df = df[df.YearRent>=2021]
# df['MonthRent'] = df['DateUpdateListing'].dt.month

df['BedBath'] = df.Beds + df.Baths
df = df[df.BedBath<=16]

df.YearBuilt = df.YearBuilt.replace(np.nan, df.YearBuilt.mode()[0]) 


# df.StateCode = df.StateCode.astype('category').cat.codes
# df.PostalCode = df.PostalCode.astype('category').cat.codes
# df.City = df.City.astype('category').cat.codes
# df.Neighborhood = df.Neighborhood.astype('category').cat.codes
# df.PropertyType = df.PropertyType.astype('category').cat.codes
# df.YearBuilt = df.YearBuilt.astype('category').cat.codes 

state = pd.read_csv('StateCodeDict.csv', header=None, index_col=0, squeeze=True).to_dict()
df.StateCode = df.StateCode.map(state)
post = pd.read_csv('PostalCodeDict.csv', header=None, index_col=0, squeeze=True).to_dict()
df.PostalCode = df.PostalCode.map(post)
city = pd.read_csv('CityDict.csv', header=None, index_col=0, squeeze=True).to_dict()
df.City = df.City.map(city)
neighborhood = pd.read_csv('NeighborhoodDict.csv', header=None, index_col=0, squeeze=True).to_dict()
df.Neighborhood = df.Neighborhood.map(neighborhood)
prop = pd.read_csv('PropertyTypeDict.csv', header=None, index_col=0, squeeze=True).to_dict()
df.PropertyType = df.PropertyType.map(prop)
year = pd.read_csv('YearBuiltDict.csv', header=None, index_col=0, squeeze=True).to_dict()
df.YearBuilt = df.YearBuilt.map(year)



# df['Has_Land'] = df['LotSize'].apply(lambda x: 1 if x > 0 else 0)
# df['Has_Size'] = df['BuildingSize'].apply(lambda x: 1 if x > 0 else 0)


cols_to_drop = ['RealtorListingId', 'RealtorPropertyId',  'DateUpdateListing', 'Address', 'BuildingSize', 'lotSize', 'YearRent']
df = df.drop(cols_to_drop, axis=1) 
gc.collect()


# newdf=df[df.groupby('PostalCode')['PostalCode'].transform('count')<5].copy()
# df = df.append([newdf]*3,ignore_index=True)

# df=df[df.groupby('PostalCode')['PostalCode'].transform('count')>5] #не влияет и при 5

# df=df[df.groupby('Neighborhood')['Neighborhood'].transform('count')>2] не влияет при 1,2

df.shape

(535823, 12)

In [ ]:
# df['MedPricePreMonth'] = df.groupby(['MonthRent'])['Price'].transform(lambda x: x.median()).round()
# df['MedPricePreMonth'].groupby(by=[df.MonthRent]).median().plot()

In [ ]:
basic_details(df)

,"Missing value, %","Zero value, %",N unique value,dtype
Price,0.0,0.0000,4293,int64
Latitude,0.0,0.0000,75088,float64
Longitude,0.0,0.0000,75173,float64
PostalCode,0.0,0.0042,9666,int16
StateCode,0.0,0.9056,49,int8
City,0.0,0.0011,6098,int16
Neighborhood,0.0,0.0021,20226,int16
PropertyType,0.0,38.8241,11,int8
YearBuilt,0.0,0.0379,189,int16
Beds,0.0,6.1047,12,int64


In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2)

# newdf=df_train[df_train.groupby('PostalCode')['PostalCode'].transform('count')<5].copy()
# df_train = df_train.append([newdf]*4,ignore_index=True)



# df_train=df_train[df_train.groupby('PostalCode')['PostalCode'].transform('count')>3]

# df_train['CenterDistance'] = round(np.sqrt((df_train.Latitude -  df_train.groupby('City')["Latitude"].transform('mean'))**2 + (df_train.Longitude -  df_train.groupby('City')["Longitude"].transform('mean'))**2)*111, 2)
# df_test['CenterDistance'] = round(np.sqrt((df_test.Latitude -  df_test.groupby('City')["Latitude"].transform('mean'))**2 + (df_test.Longitude -  df_test.groupby('City')["Longitude"].transform('mean'))**2)*111, 2)
# df_train = df_train[df_train.CenterDistance<300]


# df_train['Price_per_ZIP'] = df_train.groupby(['PostalCode'])['Price'].transform(lambda x: x.median()).round()
# df_train["ZIPFreq"] = (df_train.groupby("PostalCode")["PostalCode"].transform("count")/ df_train.PostalCode.count())*10000
# df_train['Price_per_BedBath'] = df_train.groupby('BedBath')['Price'].transform(lambda x: x.median()).round()



X_train = df_train.copy()
y_train = X_train.pop("Price")

X_test = df_test.copy()
y_test = X_test.pop("Price")

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2020)




# c = {}
# for indice_fila, x_train in X_train.iterrows():
#     c.update({x_train['PostalCode']:x_train['Price_per_ZIP']})

# X_test['Price_per_ZIP'] = 0.00
# for indice, x_test in X_test.iterrows():
#     X_test.loc[X_test.index == indice ,'Price_per_ZIP'] = c[x_test['PostalCode']]

# a = {}
# for indice_zipfreq, x_train in X_train.iterrows():
#     a.update({x_train['PostalCode']:x_train['ZIPFreq']})

# X_test['ZIPFreq'] = 0.00
# for indice_zf, x_test in X_test.iterrows():
#     X_test.loc[X_test.index == indice_zf ,'ZIPFreq'] = a[x_test['PostalCode']]

# b = {}
# for indice_bbath, x_train in X_train.iterrows():
#     b.update({x_train['BedBath']:x_train['Price_per_BedBath']})

# X_test['Price_per_BedBath'] = 0.00
# for indice_bb, x_test in X_test.iterrows():
#     X_test.loc[X_test.index == indice_bb ,'Price_per_BedBath'] = b[x_test['BedBath']]




In [ ]:
model = RandomForestRegressor(max_depth=36, n_estimators=290, n_jobs=-1, random_state=2020).fit(X_train, y_train)

scores = (-1 * cross_val_score(model, X_train, y_train,
                              cv=5,
                              scoring=my_scorer)).round()
score = mdape(y_test, model.predict(X_test))
score.median()

exception calling callback for <Future at 0x1769f249040 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "C:\Users\Dmitry\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "C:\Users\Dmitry\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\parallel.py", line 340, in __call__
    self.parallel.dispatch_next()
  File "C:\Users\Dmitry\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\parallel.py", line 768, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "C:\Users\Dmitry\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\parallel.py", line 834, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\Dmitry\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\parallel.py", line 753, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "C:\

0.11994996998428559

In [ ]:
# model = XGBRegressor(objective ='reg:squarederror', subsample=0.8, colsample_bytree = 0.8, learning_rate = 0.3,
#                 max_depth = 22, n_estimators = 250).fit(X_train, y_train)

# scores = (-1 * cross_val_score(model, X_train, y_train,
#                               cv=5,
#                               scoring='neg_mean_absolute_percentage_error')).round()
# score = mean_absolute_percentage_error(y_test, model.predict(X_test))
# score.mean()

In [ ]:
pred_av = model.predict(X_test)
df_pred = pd.DataFrame({'Actual': y_test, 'Predicted': pred_av, 'Delta': round(abs(pred_av-y_test)/y_test, 4)})
r2_score(y_test, pred_av)

0.7331382571086142

In [ ]:
df_pred.Delta.describe()

count    19015.000000
mean         0.185076
std          0.234598
min          0.000000
25%          0.051900
50%          0.119900
75%          0.240800
max          6.328200
Name: Delta, dtype: float64

In [ ]:
# cats to dict and then to csv

# df['YearBuilt'] = df['YearBuilt'].astype('category')
# my_list = df['YearBuilt'].cat.codes
# my_values = df['YearBuilt'].tolist()
# result= dict(zip(my_values,my_list))

# with open('YearBuiltDict.csv', 'w') as f:
#     for key in result.keys():
#         f.write("%s, %s\n" % (key, result[key]))

# from csv to dict and cats mapping
# input_file = pd.read_csv('StateCodeDict.csv', header=None, index_col=0, squeeze=True).to_dict()
# df.StateCode = df.StateCode.map(input_file)